In [104]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import io
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [105]:
from numpy.random import seed
seed(100)

# Loading the data

In [106]:
#!git clone https://github.com/steveshaoucsb/ML_Final_Project.git

In [107]:
main_df = pd.read_csv("/Users/steveshao/Desktop/Data_science/ML_Final_Project/Features_and_Target.csv")

In [108]:
main_df

,permno,yyyymm,ticker,r1m,close_price,AnnouncementReturn,betaVIX,BookLeverage,ConvDebt,DolVol,...,Consumer_sentiment,Date_encode,Investor_sentiment_3MA,Investor_sentiment_6MA,News_sentiment_3MA,News_sentiment_6MA,Consumer_sentiment_3MA,Consumer_sentiment_6MA,forward_return,target
0,24643,199208,AA,-11.830509,64.625,-0.034219,-0.002155,-1.991768,-1.0,-6.319875,...,0.3805,1,NaN,NaN,NaN,NaN,NaN,NaN,2.514507,1
1,24643,199209,AA,2.514507,66.250,-0.034219,-0.001269,-1.991768,-1.0,-6.289308,...,0.3780,2,NaN,NaN,NaN,NaN,NaN,NaN,5.283019,1
2,24643,199210,AA,5.283019,69.750,0.002048,-0.001911,-1.991768,-1.0,-6.103797,...,0.3665,3,0.422500,NaN,0.487161,NaN,0.375000,NaN,1.111111,1
3,24643,199211,AA,1.111111,70.125,0.002048,0.000085,-1.991768,-1.0,-5.755551,...,0.4265,4,0.481250,NaN,0.488176,NaN,0.390333,NaN,2.139037,1
4,24643,199212,AA,2.139037,71.625,0.002048,-0.006525,-1.991768,-1.0,-6.073234,...,0.4550,5,0.569167,NaN,0.530369,NaN,0.416000,NaN,3.839441,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28594,89070,201412,ZMH,1.202242,113.420,0.030118,0.000256,-1.521382,0.0,-8.063711,...,0.4680,269,0.616750,0.590042,0.566568,0.545846,0.448833,0.431833,-1.163816,0
28595,89070,201501,ZMH,-1.163816,112.100,-0.013885,0.005394,-1.521382,0.0,-7.463536,...,0.4905,270,0.622750,0.601208,0.590869,0.561545,0.467500,0.445417,7.395184,1
28596,89070,201502,ZMH,7.395184,120.390,-0.013885,-0.002619,-1.521382,0.0,-7.653775,...,0.4770,271,0.598583,0.604125,0.594520,0.571278,0.478500,0.456167,-2.201182,0
28597,89070,201503,ZMH,-2.201182,117.520,-0.013885,0.004160,-1.521382,0.0,-8.047728,...,0.4650,272,0.576917,0.596833,0.581778,0.574173,0.477500,0.463167,-6.535058,0


In [109]:
len(main_df["ticker"].unique())

137

# Preprocessing the data

In [110]:
main_df.dropna(subset=['Investor_sentiment_3MA', 'News_sentiment_3MA',
          "Consumer_sentiment_3MA", "forward_return", "betaVIX", "BookLeverage", "ConvDebt", "DolVol", "hire", 
          "IdioVol3F", "IndMom", "MaxRet", "OPLeverage", "PctAcc", "PctTotAcc", "Price", "RDIPO", "ReturnSkew", "Size", 
          "STreversal", "zerotradeAlt1"], inplace=True)

In [111]:
main_df

,permno,yyyymm,ticker,r1m,close_price,AnnouncementReturn,betaVIX,BookLeverage,ConvDebt,DolVol,...,Consumer_sentiment,Date_encode,Investor_sentiment_3MA,Investor_sentiment_6MA,News_sentiment_3MA,News_sentiment_6MA,Consumer_sentiment_3MA,Consumer_sentiment_6MA,forward_return,target
2,24643,199210,AA,5.283019,69.750,0.002048,-0.001911,-1.991768,-1.0,-6.103797,...,0.3665,3,0.422500,NaN,0.487161,NaN,0.375000,NaN,1.111111,1
3,24643,199211,AA,1.111111,70.125,0.002048,0.000085,-1.991768,-1.0,-5.755551,...,0.4265,4,0.481250,NaN,0.488176,NaN,0.390333,NaN,2.139037,1
4,24643,199212,AA,2.139037,71.625,0.002048,-0.006525,-1.991768,-1.0,-6.073234,...,0.4550,5,0.569167,NaN,0.530369,NaN,0.416000,NaN,3.839441,1
5,24643,199301,AA,3.839441,74.375,-0.034033,0.003535,-1.991768,-1.0,-6.045333,...,0.4465,6,0.635167,0.528833,0.577251,0.532206,0.442667,0.408833,-5.176470,0
6,24643,199302,AA,-5.176470,70.125,-0.034033,-0.006632,-1.991768,-1.0,-6.001115,...,0.4330,7,0.621000,0.551125,0.606878,0.547527,0.444833,0.417583,-8.021390,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28594,89070,201412,ZMH,1.202242,113.420,0.030118,0.000256,-1.521382,0.0,-8.063711,...,0.4680,269,0.616750,0.590042,0.566568,0.545846,0.448833,0.431833,-1.163816,0
28595,89070,201501,ZMH,-1.163816,112.100,-0.013885,0.005394,-1.521382,0.0,-7.463536,...,0.4905,270,0.622750,0.601208,0.590869,0.561545,0.467500,0.445417,7.395184,1
28596,89070,201502,ZMH,7.395184,120.390,-0.013885,-0.002619,-1.521382,0.0,-7.653775,...,0.4770,271,0.598583,0.604125,0.594520,0.571278,0.478500,0.456167,-2.201182,0
28597,89070,201503,ZMH,-2.201182,117.520,-0.013885,0.004160,-1.521382,0.0,-8.047728,...,0.4650,272,0.576917,0.596833,0.581778,0.574173,0.477500,0.463167,-6.535058,0


In [112]:
main_df['forward_return_div100'] = main_df['forward_return']/100

In [113]:
# Split the data into features and target
X = main_df[['Investor_sentiment_3MA', 'News_sentiment_3MA',
          "Consumer_sentiment_3MA", "betaVIX", "BookLeverage", "ConvDebt", "DolVol", "hire", 
          "IdioVol3F", "IndMom", "MaxRet", "OPLeverage", "PctAcc", "PctTotAcc", "Price", "RDIPO", "ReturnSkew", "Size", 
          "STreversal", "zerotradeAlt1"]]
y = main_df[['forward_return_div100']]

# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

# Convert the data into 3D tensors for the CNN model
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [114]:
start_year = 1992  # Start year of your dataset
training_years = 8
validation_years = 2
testing_years = 5

train_end_year = start_year + training_years - 1
valid_end_year = train_end_year + validation_years
test_end_year = valid_end_year + testing_years

# Initialize lists to store the splits
training_splits = []
validation_splits = []
testing_splits = []


In [115]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit


In [116]:
main_df['year'] = main_df['yyyymm'].astype(str).str[:4]

In [117]:
main_df['year']

2        1992
3        1992
4        1992
5        1993
6        1993
         ... 
28594    2014
28595    2015
28596    2015
28597    2015
28598    2015
Name: year, Length: 26890, dtype: object

In [118]:
main_df['year'] = main_df['year'].astype(int)

In [119]:
# 3. Roll the splits forward by 5 years at a time
while test_end_year <= 2020:  # Assuming your dataset goes up to 2020
    # Select the data for the current split
    train_data = main_df[(main_df['year'] >= start_year) & (main_df['year'] <= train_end_year)]
    valid_data = main_df[(main_df['year'] > train_end_year) & (main_df['year'] <= valid_end_year)]
    test_data = main_df[(main_df['year'] > valid_end_year) & (main_df['year'] <= test_end_year)]

    # Append the data to the respective lists
    training_splits.append(train_data)
    validation_splits.append(valid_data)
    testing_splits.append(test_data)

    # Update the start and end years for the next split
    start_year += 5
    train_end_year += 5
    valid_end_year += 5
    test_end_year += 5

In [123]:
# Define a function for creating and training a neural network model
def create_and_train_nn(X_train, y_train, X_valid, y_valid, X_test, y_test):
    # Standardize the features using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)
    X_test = scaler.transform(X_test)

    # Create a simple feedforward neural network model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32)

    # Evaluate the model on the test data
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse, model


In [124]:
trained_models = []
for train_data, valid_data, test_data in zip(training_splits, validation_splits, testing_splits):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

    mse, model = create_and_train_nn(X_train, y_train, X_valid, y_valid, X_test, y_test)

    trained_models.append(model)
    
    print(f"Mean Squared Error for this split: {mse}")

Epoch 1/50
673/673 [==============================] - 1s 871us/step - loss: 0.0183 - val_loss: 0.0103
Epoch 2/50
673/673 [==============================] - 1s 774us/step - loss: 0.0106 - val_loss: 0.0094
Epoch 3/50
673/673 [==============================] - 1s 778us/step - loss: 0.0098 - val_loss: 0.0094
Epoch 4/50
673/673 [==============================] - 1s 784us/step - loss: 0.0102 - val_loss: 0.0088
Epoch 5/50
673/673 [==============================] - 1s 791us/step - loss: 0.0093 - val_loss: 0.0092
Epoch 6/50
673/673 [==============================] - 1s 790us/step - loss: 0.0093 - val_loss: 0.0090
Epoch 7/50
673/673 [==============================] - 1s 810us/step - loss: 0.0092 - val_loss: 0.0086
Epoch 8/50
673/673 [==============================] - 1s 767us/step - loss: 0.0092 - val_loss: 0.0088
Epoch 9/50
673/673 [==============================] - 1s 764us/step - loss: 0.0090 - val_loss: 0.0084
Epoch 10/50
673/673 [==============================] - 1s 765us/step - loss: 0.008

In [135]:
trained_models[2].summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 64)                1344      
                                                                 
 dense_70 (Dense)            (None, 32)                2080      
                                                                 
 dense_71 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,457
Trainable params: 3,457
Non-trainable params: 0
_________________________________________________________________


In [133]:
from tensorflow.keras.utils import plot_model

plot_model(trained_models[0], to_file='model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


TypeError: eval() arg 1 must be a string, bytes or code object

In [137]:
pd.DataFrame(trained_models[0].history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0.1,0.2)
plt.show()

ValueError: DataFrame constructor not properly called!